In [ ]:
import imutils.paths as path
import random
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
# import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
import math
import time

In [ ]:
jumClass = 3
kernelSize = 3
filterConv1 = 32
filterConv2 = 32
neuronFLC1 = 96
neuronFLC2 = 96
# neuronFLC3 = 64
size = 32
skenario = 0.1
channel = 3
# nama='f10gray.ckpt'

In [ ]:
PATH = 'augmentasi82'
imagePaths = sorted(list(path.list_images(PATH)))
random.seed(2)
random.shuffle(imagePaths)
data = []
labels = []

In [ ]:
sess = tf.Session()
# imagePaths
# a=cv2.imread('dataset334/berat/data1-001.jpg')
# a=cv2.resize(a,(34,34))
# len(imagePaths)

In [ ]:
asda = time.time()

for i in imagePaths:
    img = cv2.imread(i)
#     plt.imshow(img)
    imageScale = cv2.resize(img,(size,size))
    imageScale = imageScale/255
#     imageScale = sess.run(tf.image.rgb_to_grayscale(imageScale))
    data.append(imageScale)
    label = i.split(os.path.sep)[-2]
    labels.append(label)
print(time.time()-asda)


In [ ]:
# PATH = 'dataset/train'
# imagePaths = sorted(list(path.list_images(PATH)))
# random.seed(2)
# random.shuffle(imagePaths)
# xTrain = []
# yTrainCls = []
# asda = time.time()
# for i in imagePaths:
#     img = cv2.imread(i)
#     imageScale = cv2.resize(img,(size,size))
#     imageScale = imageScale/255
# #     imageScale = cv2.cvtColor(imageScale, cv2.COLOR_BGR2GRAY)
# #     imageScale = sess.run(tf.image.rgb_to_grayscale(imageScale))
#     xTrain.append(imageScale)
#     label = i.split(os.path.sep)[-2]
#     yTrainCls.append(label)
# print(time.time()-asda)

In [ ]:
# PATH = 'dataset/testing'
# imagePaths = sorted(list(path.list_images(PATH)))
# random.seed(2)
# random.shuffle(imagePaths)
# xTest = []
# yTestCls = []
# asda = time.time()
# for i in imagePaths:
#     img = cv2.imread(i)
#     imageScale = cv2.resize(img,(size,size))
#     imageScale = imageScale/255
# #     imageScale = cv2.cvtColor(imageScale, cv2.COLOR_BGR2GRAY)
# #     imageScale = sess.run(tf.image.rgb_to_grayscale(imageScale))
#     xTest.append(imageScale)
#     label = i.split(os.path.sep)[-2]
#     yTestCls.append(label)
# print(time.time()-asda)

In [ ]:
dt = np.array(data)
lbl = np.array(labels)

In [ ]:
# xTrain = np.asarray(xTrain)
# yTestCls = np.asarray(yTestCls)
# yTrainCls = np.asarray(yTrainCls)
# xTest = np.asarray(xTest)
    

In [ ]:
xTrain, xTest, yTrainCls, yTestCls = train_test_split(dt, lbl, test_size=skenario)


In [ ]:
len(yTrainCls),len(yTestCls)

In [ ]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [ ]:
encoder = LabelEncoder()
encoder.fit(yTrainCls)
y = encoder.transform(yTrainCls)
yTrain = one_hot_encode(y)

In [ ]:
encoder = LabelEncoder()
encoder.fit(yTestCls)
y = encoder.transform(yTestCls)
yTest = one_hot_encode(y)

In [ ]:
x = tf.placeholder(dtype = tf.float32, shape=[None,size,size,channel], name='x')
# xImage = tf.reshape(x, shape=[-1,size,size,3])
# keepProb = tf.placeholder(dtype=tf.float32, name='keepProb')

In [ ]:
def layerKonvolusi(masukan, kernel, filterConv):
    weight = tf.Variable(tf.truncated_normal(kernel,stddev=0.05))
    konvolusi = tf.nn.conv2d(masukan, weight, strides=[1,1,1,1], padding='SAME')
    active = tf.nn.relu(konvolusi)
#     minimal = tf.math.reduce_min(konvolusi)
#     maximal = tf.math.reduce_max(konvolusi)
#     s1 = konvolusi-minimal
#     s2 = maximal-minimal
#     active = s1/s2
#     print(active)
    maxpool = tf.nn.max_pool(active, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    return maxpool

In [ ]:
outConv1 = layerKonvolusi(x, kernel=[kernelSize,kernelSize,channel, filterConv1], filterConv=filterConv1)

In [ ]:
outConv2 = layerKonvolusi(outConv1, kernel=[kernelSize,kernelSize,filterConv1,filterConv2], filterConv=filterConv2)

In [ ]:
def layerNN(masukan, nInput, nOutput, dropRelu):
    weight = tf.Variable(tf.truncated_normal(shape=[nInput,nOutput], stddev=0.05))
    bias = tf.Variable(tf.constant(1.0,shape=[nOutput]))
    fcl = tf.matmul(masukan, weight)
    out = fcl+bias
    
    if dropRelu:
        out = tf.nn.relu(out)
        out = tf.nn.dropout(out,0.2)
        
    return out

In [ ]:
indexFlat = math.ceil(size/4)
outFlat = tf.reshape(outConv2, shape=[-1, indexFlat*indexFlat*filterConv2])
outHL1 = layerNN(outFlat, indexFlat*indexFlat*filterConv2, neuronFLC1, True)
outHL2 = layerNN(outHL1, neuronFLC1, neuronFLC2, True)
# outHL3 = layerNN(outHL2, neuronFLC2, neuronFLC3, True)

In [ ]:
y = layerNN(outHL2, neuronFLC2, jumClass, False)

yPred = tf.nn.softmax(y, name='yPred')

In [ ]:
y_ = tf.placeholder(tf.float32,shape=[None,jumClass], name='y_')

In [ ]:
yPredArg = tf.argmax(yPred, dimension=1)
y_Arg = tf.argmax(y_, dimension=1)
yArg = tf.argmax(y, dimension=1)

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y,labels=y_))
correctPrediction = tf.equal(yPredArg, y_Arg)
accuracyPrediction = tf.reduce_mean(tf.cast(correctPrediction,tf.float32))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
yPreda = []
ya = []
y_a = []
fitur = []
lab = []

In [ ]:
saver = tf.train.Saver()

In [ ]:
start = time.time()
for i in range(100):
    for k in range(math.ceil(len(xTrain)/20)):
        xBatch = xTrain[k*20:min((k+1)*20,len(xTrain))]
        yBatch = yTrain[k*20:min((k+1)*20,len(yTrain))]
#         print(yTrain[1])
        sess.run(optimizer, feed_dict={
            x:xBatch,
            y_:yBatch
#             keepProb:0.2
        })
    
        flat,labelnya, loss, acc= sess.run([outFlat,y_, cost,accuracyPrediction], feed_dict={
            x:xBatch,
            y_:yBatch
#             keepProb:0.2
        })
        
#         if(i==99):
#             fitur.append(flat)
#             lab.append(labelnya)
            
    tesAcc, tesLoss, tesY, labelY, softY = sess.run([accuracyPrediction,cost, yArg, y_Arg, yPredArg], feed_dict={
        x:xTest,
        y_:yTest
#         keepProb:0.2
    })
    
    print("Iter " + str(i) + ", Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc) +
         ", Loss Test " + "{:.6f}".format(tesLoss) + ", Accuration Test " + "{:.5f}".format(tesAcc))
    
    saver.save(sess, "/home/athed/Dropbox/skripsi/project/model/fix")
print(time.time()-start,"s")
ya=tesY
y_a=labelY
yPreda=softY

In [ ]:
# saver.save(sess, "/home/athed/Dropbox/skripsi/project/model/fix")

In [ ]:
ppp= sess.run(tf.confusion_matrix(labels=y_a,predictions=yPreda,num_classes=3, name='s'))
print(ppp)

In [33]:
data=[]
img = cv2.imread("/home/athed/Dropbox/skripsi/project/augmentasi82/berat/data1-001.jpg")
#     plt.imshow(img)
imageScale = cv2.resize(img,(32,32))
imageScale = imageScale/255
#     imageScale = sess.run(tf.image.rgb_to_grayscale(imageScale))
data.append(imageScale)
print(sess.run(yPred,feed_dict={
    x:data
#     keepProb:0.2
}))

[[9.9998128e-01 4.1043520e-09 1.8657012e-05]]


In [46]:
saver = tf.train.import_meta_graph('model/fix.meta')
saver.restore(sess, tf.train.latest_checkpoint('model/'))
graph = tf.get_default_graph()
p= graph.get_tensor_by_name("x:0") 
yPreda = graph.get_tensor_by_name("yPred:0")
aa=sess.run(yPreda,feed_dict={
    p:data
#     keepProb:0.2
})
print(aa)
print(np.argmax(aa))

INFO:tensorflow:Restoring parameters from /home/athed/Dropbox/skripsi/project/model/fix
[[9.9977392e-01 1.2708865e-07 2.2592797e-04]]
0


In [ ]:
a=0
# print(sess.run(yPredArg, feed_dict={
#     x:xTest,
#     keepProb:0.2
# }))
# print(yTestCls)
xBatch = xTrain[a:(a+1)]
kelas = yTestCls[a:(a+1)]
gambar = xTest[a:(a+1)]

# print(sess.run(y, feed_dict={
#     x:xBatch,
#     keepProb:0.2
# }))
# aya= sess.run(yPred, feed_dict={
#     x:xBatch,
#     keepProb:0.2
# })
# print(aya,np.argmax(aya))
print(sess.run(y,feed_dict={
    x:xBatch,
    keepProb:0.2
}))
# plt.title(kelas[0])
# plt.imshow(gambar[0])

In [ ]:
my_graph = my_graph = tf.get_default_graph()
tf.train.write_graph(my_graph,"/home/athed/Dropbox/skripsi/project/model",
                     '/model/saved_model.pb', as_text=False)

In [ ]:
# y_a=np.asarray(y_a)
# print(y_a)
# y_aa = [2,2,2,1,0,1,1,1,0]
# print(type(y_aa))
# labelnya = np.asarray(labelnya)
# labelnya.shape
# np.savetxt('a.txt',flat, fmt="%7.3f")
# aaa = sess.run(y_Arg, feed_dict={
#     y_:yTest
# })
fitur = sess.run(outFlat, feed_dict={
    x:xTest,
    y:yTest,
    keepProb:0.2
})

In [ ]:
fitur = np.asarray(fitur)
fitur.shape
# print(fitur)

In [ ]:
nnn = len(yTestCls)
len(yTestCls)

In [ ]:
clsn=[]
berat=[]
nn=[]
nama = "sedang"
for i in range (2048):
    x=0
    clsn = 0
#     for j in range (20):
    for k in range(nnn):
        if yTestCls[k] == nama:
            x = x + fitur[k,i]
            clsn = clsn + 1 
    nn.append(x)    
    n="gempa-fitur-"+nama+".xls"
    np.savetxt(n,nn,fmt="%.3f")
print(clsn)

In [ ]:
clsn=[]
berat=[]
nn=[]
nama = "ringan"
for i in range (2048):
    x=0
    clsn = 0
#     for j in range (20):
    for k in range(nnn):
        if yTestCls[k] == nama:
            x = x + fitur[k,i]
            clsn = clsn + 1 
    nn.append(x)    
    n="gempa-fitur-"+nama+".xls"
    np.savetxt(n,nn,fmt="%.3f")
print(clsn)

In [ ]:
clsn=[]
berat=[]
nn=[]
nama = "berat"
for i in range (2048):
    x=0
    clsn = 0
#     for j in range (20):
    for k in range(nnn):
        if yTestCls[k] == nama:
            x = x + fitur[k,i]
            clsn = clsn + 1 
    nn.append(x)    
    n="gempa-fitur-"+nama+".xls"
    np.savetxt(n,nn,fmt="%.3f")
print(clsn)

In [ ]:
nn = np.asarray(nn)
nn.shape

In [ ]:
data = fitur
data.shape
# print(data[1,:,:])
for i in range(42):
    n="fitur/fitur"+str(i)+".xls"
    np.savetxt(n,data[i,:,:],fmt="%.3f")
    

In [ ]:


# Write the array to disk
with open('test.txt', 'w') as outfile:
    # I'm writing a header here just for the sake of readability
    # Any line starting with "#" will be ignored by numpy.loadtxt
    outfile.write('# Array shape: {0}\n'.format(data.shape))

    # Iterating through a ndimensional array produces slices along
    # the last axis. This is equivalent to data[i,:,:] in this case
    for data_slice in data:

        # The formatting string indicates that I'm writing out
        # the values in left-justified columns 7 characters in width
        # with 2 decimal places.  
        np.savetxt(outfile, data_slice, fmt='%-7.2f')

        # Writing out a break to indicate different slices...
        outfile.write('# New slice\n')

In [ ]:
# print("ntp\n" , nTP, "stp\n",st)
# yPreda = np.asarray(yPreda)
# print(yPreda)
# nn=sess.run(tf.reshape(yPreda,[1]))
# ppp= sess.run(tf.confusion_m?matrix(labels=y_a,predictions=yPreda,num_classes=2, name='s')))

In [ ]:
# plt.figure(figsize=(8, 6))
# plt.imshow(ppp, interpolation='nearest', cmap=None)
# plt.title("title")
# plt.colorbar()

In [ ]:
# print(type(yTestCls))
# nn = np.array(ya.tolist())
# print(type(y_a))
# np.sum(ppp)

In [ ]:
# aaa=np.trace(ppp)/np.sum(ppp)
# aaa

In [ ]:
d=[[10,2,3],[4,5,6]]
d=np.asarray(d)
d-=9
a=tf.nn.relu(d)
print(a)
print((sess.run(tf.nn.relu(d))))

In [ ]:
mina = sess.run(tf.math.reduce_min(d))
print(mina)
maxa = sess.run(tf.math.reduce_max(d))
print(maxa)
print(maxa-mina)

In [ ]:
print(tf.divide((d-mina),(maxa-mina)))
dm = d-mina
print("")
print(dm)
mm = maxa-mina
print("")
print(dm/mm)

# print((k))

In [ ]:
# print(tf.divide(d,2))